# Install 

In [1]:
! pip install transformers datasets evaluate
! pip install sacrebleu
! pip install googletrans==3.1.0a0
# ! pip install huggingface_hub


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [2]:
from datasets import load_dataset, DatasetDict, Dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import pipeline
from googletrans import Translator

C:\Users\ACER\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading Dataset

In [3]:
dataset = load_dataset("aslg_pc12")
dataset = dataset["train"].train_test_split(train_size=0.8)

Found cached dataset aslg_pc12 (C:/Users/ACER/.cache/huggingface/datasets/aslg_pc12/default/0.0.1/7ae5d117644e44ff4d2233f27f1d5df93f429efab7b2ea84828516fcbce48fb4)
100%|██████████| 1/1 [00:00<00:00, 13.62it/s]


# Training Model

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
source_lang = "gloss"
target_lang = "text"


def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs,  text_target=targets, max_length=128, truncation=True)
    print(type(model_inputs))
    return model_inputs

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [ ]:
sacrebleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
#trainer.push_to_hub()

# Load Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("junowhite/transformer_model")

model = AutoModelForSeq2SeqLM.from_pretrained("junowhite/transformer_model")

In [5]:
translators = Translator()

In [6]:
translator = pipeline("translation", model=model, tokenizer = tokenizer)

prediction = []
answer = []

def testing_data():
  for i in range(7):
    text = dataset["train"][i]["gloss"]
    answer.append(dataset["train"][i]["text"])
    prediction.append(translators.translate(translator(text)[0]["translation_text"]).text)

d:\Software\Python\lib\site-packages\transformers\pipelines\__init__.py:858: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [7]:
testing_data()
print(prediction)
print(answer)

['c we have ended the entire day discussing a decent strategic framework for the european union .', 'this is a great difference .', 'i will therefore be speaking on him about this incident .', 'i particularly recall your resolution of july 2006 and the written declaration of the september 200ORIGIN mark should therefore be reflected in that spirit .', 'they cannot .', 'it is undoubtedly why .', 'SWIFT debate']
['cs we have spent the entire day discussing a desirable strategic focus for the european union .\n', 'this is the great difference .\n', 'i will therefore be speaking on his behalf this evening .\n', 'i particularly recall your resolution of july 2006 and the written declaration of september 200origin marking should therefore be considered in that spirit .\n', 'they cannot .\n', 'it is understandable why .\n', 'swift debate \n']
